In [3]:
import graph_tool.all as gt

import numpy as np
import scipy as sp
import scipy.stats as sps

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
%autosave 15

Autosaving every 15 seconds


Сгенерируем случайные графы в модели Эрдёша-Реньи (ну, почти; в graph-tool ничего более похожего сделать не удалось):

In [110]:
n, p = 10, 0.5
G = gt.random_graph(n, lambda: np.random.poisson(n * p), directed=False);

Подсчитаем матрицу кратчайших расстояний алгоритмом Флойда.

In [111]:
w = G.new_edge_property("double")
w.a = np.random.rand(G.num_edges()) * 42
sd = gt.shortest_distance(G, weights=w, directed=False, dense=True)
sd_a = np.vstack([
    [sd[v] for v in G.vertices()]
])

Выделим мин.остов в подграфе, индуцированном терминальными вершинами.

In [112]:
KG = gt.complete_graph(n, directed=False)
sd_w = KG.new_edge_property("double")
for e in KG.edges():
    u, v = int(e.source()), int(e.target())
    sd_w[e] = sd_a[u, v]
V0 = np.array([1, 3, 7, 9])
V0_vfilt = G.new_vertex_property("bool")
V0_vfilt.a[V0] = True
KG_V0_induced = gt.GraphView(KG, vfilt=V0_vfilt)
KG_V0_w = KG_V0_induced.new_vertex_property("double")
MST = gt.min_spanning_tree(KG_V0_induced, weights=sd_w)

Подсчитаем его вес:

In [125]:
mst_cost = 0
for e in KG_V0_induced.edges():
    if MST[e]:
        mst_cost += sd_w[e]
print(mst_cost)

20.424116122809174


Сравним с весом реального дерева Штейнера (а как его подсчитать?):

In [126]:
paths = gt.all_paths(G, source=V0[0], target=V0[-1])
mincost = np.inf
for path in paths:
    if np.all(np.isin(V0, path)):
        cost = 0
        for i in range(len(path) - 1):
            cost += w[G.edge(path[i], path[i+1])]
        mincost = min(mincost, cost)
print(mincost)

57.578846045568596
